In [1]:
#import tensorflow as tf
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import mean_squared_error

In [14]:
def def_final(d1, d2, d3, d4, d5, d6, d7, d8):
    data = pd.read_csv('data_fin_.csv',encoding = 'cp949')
    user = pd.read_csv('user_data.csv',encoding = 'cp949')
    user['rating'] = [d1, d2, d3, d4, d5, d6, d7, d8]
    new = data[['userid','courseid','rating']]
    user_new = user[['userid','courseid','rating']]
    new = new.append(user_new)
    data_matrix = new.pivot_table('rating', index = 'userid', columns = 'courseid')
    data_matrix = data[['userid','course','rating']]
    user_matrix = user[['userid','course','rating']]
    data_matrix = data_matrix.append(user_matrix)
    rating_matrix = data_matrix.pivot_table('rating', index = 'userid', columns = 'course')
    rating_matrix = rating_matrix.fillna(0)
    rating_matrix_T = rating_matrix.transpose()
    item_sim = cosine_similarity(rating_matrix_T, rating_matrix_T)
    item_sim_df = pd.DataFrame(data = item_sim, index = rating_matrix.columns, columns = rating_matrix.columns)
    def predict_rating(rating_arr, item_sim_arr):
        rating_pred = rating_arr.dot(item_sim_arr) / np.array([np.abs(item_sim_arr).sum(axis = 1)])
        return rating_pred
    rating_pred = predict_rating(rating_matrix.values, item_sim_df.values)
    rating_pred_matrix = pd.DataFrame(data = rating_pred, index = rating_matrix.index, columns = rating_matrix.columns)
    def get_mse(pred, actual):
        pred = pred[actual.nonzero()].flatten() 
        actual = actual[actual.nonzero()].flatten()
        return mean_squared_error(pred, actual)
    def predict_rating_topsim(rating_arr, item_sim_arr, n = 20):
        pred = np.zeros(rating_arr.shape) 
    
        for col in range(rating_arr.shape[1]): 
            top_n_items = [np.argsort(item_sim_arr[:, col])[: -n-1 : -1]] 
            for row in range(rating_arr.shape[0]): 
                pred[row, col] = item_sim_arr[col, :][top_n_items].dot(rating_arr[row, :][top_n_items].T)
                pred[row, col] /= np.sum(np.abs(item_sim_arr[col, :][top_n_items]))
        return pred
    rating_pred = predict_rating_topsim(rating_matrix.values, item_sim_df.values, n = 20)
    rating_pred_matrix = pd.DataFrame(data = rating_pred, index = rating_matrix.index, columns = rating_matrix.columns)
    user_rating_id = rating_matrix.loc[8, :]
    def get_unseen_course1(rating_matrix, userid):
        user_rating = rating_matrix.loc[userid, :] 
        already_go = user_rating[ user_rating > 0 ].index.tolist()
        course_list = rating_matrix.columns.tolist()
        un_list = [ course for course in course_list if course not in already_go ]
        return un_list
    def recomm_course_by_userid1(pred_df, userid, un_list, top_n):
        recomm_course = pred_df.loc[userid, un_list].sort_values(ascending = False)[: top_n]
        return recomm_course
    un_list = get_unseen_course1(rating_matrix, 8)
    recomm_course = recomm_course_by_userid1(rating_pred_matrix, 8, un_list, 30)
    recomm_course = pd.DataFrame(data = recomm_course.values, index = recomm_course.index, columns = ['pred_score'])
    def get_rmse(R, P, Q, non_zeros):
        error = 0
        full_pred_matrix = np.dot(P, Q.T) 
        x_non_zero_ind = [non_zero[0] for non_zero in non_zeros]
        y_non_zero_ind = [non_zero[1] for non_zero in non_zeros]
        R_non_zeros = R[x_non_zero_ind, y_non_zero_ind]
        full_pred_matrix_non_zeros = full_pred_matrix[x_non_zero_ind, y_non_zero_ind]
        mse = mean_squared_error(R_non_zeros, full_pred_matrix_non_zeros)
        rmse = np.sqrt(mse)
        return rmse
    def matrix_factorization(R, K, steps, learning_rate, r_lambda):
        num_users, num_items = R.shape
        np.random.seed(1)
        P = np.random.normal(scale = 1./K, size = (num_users, K))
        Q = np.random.normal(scale = 1./K, size = (num_items, K))
    
        prev_rmse = 10000
        break_count = 0
    
        non_zeros = [ (i, j, R[i,j]) for i in range(num_users) for j in range(num_items) if R[i, j] > 0]
     
        for step in range(steps):
            for i, j, r in non_zeros:
                eij = r - np.dot(P[i, :], Q[j, :].T)
                P[i, :] = P[i, :] + learning_rate * (eij * Q[j, :] - r_lambda * P[i, :])
                Q[j, :] = Q[j, :] + learning_rate * (eij * P[i, :] - r_lambda * Q[j, :])
            rmse = get_rmse(R, P, Q, non_zeros)
            #if(step % 10) == 0:
                #print("### iteration step : ",step, " rmse : ", rmse)
        return P,Q
    rating = new[['userid', 'courseid', 'rating']]
    rating_matrix = rating.pivot_table('rating', index = 'userid', columns = 'courseid')
    rating_matrix = data_matrix.pivot_table('rating', index = 'userid', columns = 'course')
    P, Q = matrix_factorization(rating_matrix.values, 50, 100, 0.01, 0.01)
    pred_matrix = np.dot(P, Q.T)
    rating_pred_matrix = pd.DataFrame(data = pred_matrix, index = rating_matrix.index, columns = rating_matrix.columns)
    def get_unseen_course2(rating_matrix, userid):
        user_rating = rating_matrix.loc[userid, :]
        already_go = user_rating[ user_rating > 0 ].index.tolist()
        course_list = rating_matrix.columns.tolist()
        un_list = [ course for course in course_list if course not in already_go ]
        return un_list
    def recomm_course_by_userid2(pred_df, userid, un_list, top_n):
        recomm_course = pred_df.loc[userid, un_list].sort_values(ascending = False)[: top_n]
        return recomm_course
    un_list = get_unseen_course2(rating_matrix, 8)
    matrix_recomm_course = recomm_course_by_userid2(rating_pred_matrix, 8, un_list, 30)
    matrix_recomm_course = pd.DataFrame(data = matrix_recomm_course.values, index = matrix_recomm_course.index, columns = ['pred_score'])
    course_df = data.drop(['userid', 'rating'], axis = 1)
    user_df = user[['course','rating']]
    inputId = course_df[course_df['course'].isin(user_df['course'].tolist())]
    result = pd.merge(inputId, user_df)
    result_df = result[['courseid', 'course', 'rating']]
    result_detail = result[['specific', 'water', 'tree', 'ruins', 'eat', 'hotel', 'cafe', 'pet', 'spring', 'summer', 'fall', 'winter']]
    userProfile = result_detail.transpose().dot(result_df['rating'])
    courseTable = course_df.set_index(course_df['courseid'])
    courseTable = courseTable[['specific', 'water', 'tree', 'ruins', 'eat', 'hotel', 'cafe', 'pet', 'spring', 'summer', 'fall', 'winter']]
    recoTable_df = ((courseTable * userProfile).sum(axis = 1)) / (userProfile.sum())
    recoTable_df = recoTable_df.sort_values(ascending = False)
    rank = course_df.loc[course_df['courseid'].isin(recoTable_df.head(20).keys())]
    rankId = data[data['course'].isin(rank['course'].tolist())]
    final = pd.merge(rankId, rank)
    fi_result = final[['courseid', 'course', 'rating']].sort_values('rating', ascending = False)
    fi_result = fi_result.drop_duplicates('courseid', keep='first')
    fi_result = fi_result.reset_index(drop = True)
    #fi_result.to_csv("content_based.csv")
    recomm_course.to_csv("item_based.csv")
    matrix_recomm_course.to_csv("matrix_factorization.csv")
    item_based = pd.read_csv('item_based.csv', encoding = 'utf-8')
    matrix_factorization = pd.read_csv('matrix_factorization.csv', encoding = 'utf-8')
    dupli = item_based[item_based['course'].isin(matrix_factorization['course'].tolist())]
    dupli = dupli.reset_index(drop = True)
    return matrix_factorization, fi_result

In [15]:
dupli, fi_result = def_final(8, 7, 2, 3, 5, 7, 8, 7)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:32: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:33: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.


In [16]:
dupli # matrix_factorization & item_based 

,course,pred_score
0,"제주협재해수욕장,오설록티뮤지엄,폴부엌,협재식물원카페,나홀로나무,새별오름",7.374845
1,"군산근대문화역사박물관,안젤라분식,초원사진관,히로쓰가옥,고우당,테디베어뮤지엄,이성당,...",7.143892
2,"여의도한강공원,익선동,북촌한옥마을,인사동,청계천,광장시장",7.109864
3,블루로드 : 영덕 대게공원~고래불해수욕장,7.076712
4,"제주애월백번가든,협재해수욕장,애월더선셋,김녕미로공원,만장굴,코코마마",6.880873
5,"인천대공원,현대프리미엄아울렛송도점,오크우드프리미어인천오크레스토랑",6.780705
6,"여수청정게장촌,카페듀,한화아쿠아플라넷,오동도,여수케이블카,청해식당",6.773029
7,"제주지금여기,네거리식당,서귀포매일올레시장,올레은하수마카롱,한림바다생태체험마을",6.752317
8,"유진게찜,엘꼬시네로(스페인음식)",6.748305
9,"전주투모루오카페,베테랑칼국수,전주한옥마을",6.745011


In [13]:
fi_result #content_based 결과

,courseid,course,rating
0,582,"부산남포동부산길거리음식,해운대더베이101",10
1,592,"곡성씨엘로,가정역폐역,도림사계곡,단편카페",10
2,769,"군산은파호수공원,지린성,초원사진관,군산과자조합,카페199,경일철도마을,선유도해수욕장...",10
3,661,"여수돌산공원,여수케이블카,오동도,향일암,카페작금,여진식당",9
4,741,"강릉해미가,오죽헌,경포해변,테라로사사천점,초당등불생등심,cafe106",8
5,733,"단양도담상봉,석문,다누리아쿠아리움,구경시장,만천하스카이워크,잔도길",8
6,748,"강릉경포해변,초당114,허균허난설헌기념공원,경포호,원조강릉교동반점",7
7,737,"군산일제강점기역사박물관,올드브릭카페,근대역사박물관,근대건축박물관,해양공원,작은떡시루...",7
